### FOUILLET Elena / HARTMANN Louise

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import shutil
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# 1. Récupération et affichages des données 

In [ ]:
# Liste des catégories de bateaux
categories = [
    "coastguard_scaled",
    "containership_scaled",
    "corvette_scaled",
    "cruiser_scaled",
    "cv_scaled",
    "destroyer_scaled",
    "methanier_scaled",
    "smallfish_scaled",
    "submarine_scaled",
    "tug_scaled"
]

Avant de nous plonger dans le défi de la détection du type de bateau à partir d'images, il est essentiel de passer par une étape cruciale : le prétraitement des données. Cette étape revêt une grande importance dans la construction d'un réseau de neurones performant.

Nos données consistent en un ensemble d'images de bateaux, chacune ayant une taille de 16x24 pixels. Ces images peuvent contenir des variations de couleurs, de luminosité, de rotation et d'autres facteurs qui peuvent impacter la capacité de notre modèle à apprendre des caractéristiques discriminantes.

Dans le cadre du prétraitement des données, nous allons appliquer différentes techniques pour améliorer la qualité et la pertinence de nos images. Cela peut inclure des opérations telles que l'ajustement de la luminosité, le contraste, la netteté, la rotation, la mise à l'échelle, etc.

En visualisant ces images, nous pourrons mieux comprendre la diversité et les défis potentiels associés à notre ensemble de données. Cela nous permettra également de nous familiariser avec les différents types de bateaux présents dans nos images, ce qui sera utile lors de l'analyse des performances de notre modèle par la suite.

Passons maintenant à l'affichage de ces images, afin de pouvoir les observer et en apprendre davantage sur notre ensemble de données.

In [ ]:
#Afficher les images de bateaux
import matplotlib.pyplot as plt
from PIL import Image

size_img = (16,24)

path = "/kaggle/input/navires-2023-la-mano/ships16x24/ships_16x24_10cat/data/"
fig = plt.figure(figsize=(15, 7))
for i, category in enumerate(categories):
    category_dir = os.path.join(path, category)
    image_file = os.listdir(category_dir)
    img = Image.open(path + category + '/' + image_file[0])
    fig.add_subplot(3, 4, i + 1)
    plt.axis('off')
    plt.imshow(img)
    plt.title(category) 

Nous avons affiché une image représentant chaque type de bateau. Cependant, il est difficile de déterminer visuellement le type de bateau en raison du flou des images. Pour améliorer la netteté de ces photos, nous allons essayer de les ajuster grâce à la méthode de LaPlace.

In [ ]:
#Afficher les images de bateaux
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import laplace

size_img = (16,24)
sharp_filter = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
input_dir = "/kaggle/input/navires-2023-la-mano/ships16x24/ships_16x24_10cat/data/"
fig = plt.figure(figsize=(15, 7))
for i, category in enumerate(categories):
    category_dir = os.path.join(input_dir, category)
    image_file = os.listdir(category_dir)
    img_path = os.path.join(category_dir, image_file[0])
    
    img_normal = Image.open(img_path)
    img_array = np.array(img_normal)
    sharpened_array = img_array + laplace(img_array)
    
    fig.add_subplot(3, 4, i + 1)
    plt.axis('off')
    plt.imshow(sharpened_array)
    plt.title(category) 

Malheureusement, les ajustements effectués n'ont pas réussi à rendre les images plus nettes. Nous devons donc essayer une autre approche pour améliorer la qualité des données.

In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

size_img = (16, 24)
sharp_filter = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])

path = "/kaggle/input/navires-2023-la-mano/ships16x24/ships_16x24_10cat/data/"
fig = plt.figure(figsize=(15, 7))

for i, category in enumerate(categories):
    category_dir = os.path.join(input_dir, category)
    image_file = os.listdir(category_dir)
    img_path = os.path.join(category_dir, image_file[0])
    
    img_normal = Image.open(img_path)
    img_array = np.array(img_normal)
    
    # Appliquer le filtre de netteté en utilisant la convolution
    sharpened_array = cv2.filter2D(img_array, -1, sharp_filter)
    
    fig.add_subplot(3, 4, i + 1)
    plt.axis('off')
    plt.imshow(sharpened_array)
    plt.title(category)

plt.show()


Malgré les tentatives précédentes, nous constatons que les images ne sont pas significativement plus nettes. Nous devons donc trouver une autre méthode ou une combinaison d'ajustements pour améliorer la netteté des images et rendre les détails plus visibles.

In [ ]:
from PIL import Image, ImageEnhance

size_img = (240, 160)  # Nouvelle taille des images agrandies

path = "/kaggle/input/navires-2023-la-mano/ships16x24/ships_16x24_10cat/data/"
fig = plt.figure(figsize=(15, 7))

for i, category in enumerate(categories):
    category_dir = os.path.join(input_dir, category)
    image_file = os.listdir(category_dir)
    img_path = os.path.join(category_dir, image_file[0])
    
    img_normal = Image.open(img_path)
    
    # Redimensionner l'image en utilisant l'interpolation
    img_resized = img_normal.resize(size_img, Image.LANCZOS)
    
    enhancer = ImageEnhance.Sharpness(img_resized)
    img_enhanced = enhancer.enhance(2.0)
    
    fig.add_subplot(3, 4, i + 1)
    plt.axis('off')
    plt.imshow(img_enhanced)
    plt.title(category)

plt.show()

En utilisant une approche de redimensionnement des images avec un facteur d'agrandissement de 10 et en appliquant des techniques pour améliorer la netteté, nous avons réussi à obtenir des résultats plus satisfaisants. Les images agrandies et améliorées ont permis de rendre les détails plus visibles et d'améliorer la qualité générale des données.

Nous allons procéder à la division de nos données en deux dossiers distincts, à savoir "train" et "validation", en respectant une répartition de 80% pour l'entraînement et 20% pour la validation. Cela nous permettra d'obtenir deux ensembles de données distincts pour ces deux étapes essentielles de notre processus.

## 1.1 Répartition des données

In [ ]:
import os
import shutil

# Chemin du répertoire à supprimer
directory = '/kaggle/working/'

for root, dirs, files in os.walk(directory):
    for file in files:
        file_path = os.path.join(root, file)
        os.remove(file_path)
    for dir in dirs:
        dir_path = os.path.join(root, dir)
        shutil.rmtree(dir_path)
print("Tous les dossiers et fichiers ont été supprimés.")

In [ ]:
# Chemins d'accès aux répertoires
input_dir = "/kaggle/input/navires-2023-la-mano/ships16x24/ships_16x24_10cat/data"
output_dir = "/kaggle/working/ships"
train_dir = os.path.join(output_dir, "train")
val_dir = os.path.join(output_dir, "validation")
test_dir = os.path.join(output_dir, "test")

In [ ]:
# Création des répertoires de destination
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [ ]:
train_percentage = 0.8
val_percentage = 0.2

In [ ]:
# Déplacement des images vers les répertoires de destination
for category in categories:
    category_dir = os.path.join(input_dir, category)
    image_files = os.listdir(category_dir)
    
    for file in image_files:
        source_path = os.path.join(category_dir, file)
        
        if np.random.uniform() < train_percentage:
            dest_dir = os.path.join(train_dir, category)
        else:
            dest_dir = os.path.join(val_dir, category)
        
        os.makedirs(dest_dir, exist_ok=True)  # Créer le répertoire de destination s'il n'existe pas déjà
        
        destination_path = os.path.join(dest_dir, file)
        shutil.copyfile(source_path, destination_path)

# 1.2 Pré-traitement des images de train

Pour améliorer la capacité de notre réseau à interpréter les images, nous allons commencer par les agrandir. En augmentant la taille des images, nous permettons au réseau de neurones de capturer plus de détails et de mieux discerner les caractéristiques distinctives des bateaux. Cela facilitera ainsi la tâche de classification pour notre réseau.

In [ ]:
#train_dir = "/kaggle/working/ships/train"

# Parcourir les sous-dossiers
#for categorie in categories:
    # Chemin complet du sous-dossier
    #categorie_path = os.path.join(train_dir, categorie)
    
    # Liste des fichiers dans le sous-dossier
    #files = os.listdir(categorie_path)
    
    # Parcourir les fichiers
    #for file in files:
        # Chemin complet du fichier
        #file_path = os.path.join(categorie_path, file)
        
        # Ouvrir l'image avec PIL
        #image = Image.open(file_path)
        
        # Redimensionner l'image
        #resized_image = image.resize((240, 160))
        
        # Écraser l'image d'origine avec l'image redimensionnée
        #resized_image.save(file_path)

#print("Redimensionnement terminé.")

Ensuite, nous allons appliquer un filtre de netteté afin de diversifier notre jeu de données et d'améliorer la robustesse de notre modèle :

- **Netteté** : Pour améliorer la netteté des images, nous allons appliquer une technique de renforcement des contours. Cela permettra de rendre les contours des bateaux plus prononcés et de mieux mettre en évidence les détails importants.

Nous espérons augmenter la diversité de notre jeu de données et permettre à notre modèle de mieux généraliser et de détecter efficacement le type de bateau présent dans les images.

In [ ]:
import random
from PIL import ImageEnhance
from scipy.ndimage.filters import laplace

# Parcourir les sous-dossiers
for categorie in categories:
    categorie_path = os.path.join(train_dir, categorie)
    
    files = os.listdir(categorie_path)
    
    for file in files:
        file_path = os.path.join(categorie_path, file)
        
        image = Image.open(file_path)
        
        for i in range(1):
            # Améliorer la netteté
            enhancer = ImageEnhance.Sharpness(image)
            img_enhanced = enhancer.enhance(2.0)

            output_filename = f"augmented_{i}_{file}"
            output_path = os.path.join(categorie_path, output_filename)
            
            # Enregistrer l'image augmentée
            img_enhanced.save(output_path)
            
            # Afficher l'image
            #plt.imshow(img_enhanced)
            #plt.axis('off')
            #plt.title(f"Augmented Image {i+1}")
            #plt.show()

print("Augmentation des données terminée.")

In [ ]:
def count_files(folder_path):
    count = 0
    
    for root, dirs, files in os.walk(folder_path):
        count += len(files)
    
    return count

input_folder = "/kaggle/working/ships/train"

total_files = count_files(input_folder)
print(f"Nombre total de fichiers : {total_files}")

Nous avons augmenté notre jeu de données initial en créant de nouvelles images. Affichons ces nouvelles images.

In [ ]:
import os

directory = "/kaggle/working/ships/train/submarine_scaled"

files = os.listdir(directory)
for i, filename in enumerate(files[:2]):
    file_path = os.path.join(directory, filename)
    
    if os.path.isfile(file_path):
        print(filename)

In [ ]:
image1 = Image.open("/kaggle/working/ships/train/submarine_scaled/2567.jpg")
image2 = Image.open("/kaggle/input/navires-2023-la-mano/ships16x24/ships_16x24_10cat/data/submarine_scaled/2567.jpg")
image3 = Image.open("/kaggle/working/ships/train/submarine_scaled/augmented_0_2567.jpg")

fig, axes = plt.subplots(1, 3, figsize=(10, 5))
axes[0].imshow(image2) 
axes[0].axis('off')
axes[0].set_title('Image Originale')

axes[1].imshow(image1) 
axes[1].axis('off')
axes[1].set_title('Image Agrandi')

axes[2].imshow(image3) 
axes[2].axis('off')
axes[2].set_title('Variations de limage')

plt.show()

# 2. Création de notre modèle

In [ ]:
# Affichage du nombre de couches
model = Sequential()

#model.add(Input(shape=(16,24,3)))

#model.add(Rescaling(1./255))
model.add(Resizing(64,96))

# model.add(Rescaling(1./255))
# model.add(Resizing(160,240))

# Bloc 1
model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(64, 96, 3)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D())

model.add(BatchNormalization())

# Bloc 2
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(BatchNormalization())

model.add(Dropout(0.25))

# Bloc 3
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(BatchNormalization())

model.add(Dropout(0.25))

model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D())
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.build((None, 64, 96, 3))

print("Nombre de couches :", len(model.layers))

# 3. Entrainement du réseau

In [ ]:
# Paramètres d'entraînement
batch_size = 64
epochs = 25

# Générateurs de données
train_datagen = ImageDataGenerator(
    validation_split=0.2, 
    rotation_range = 10,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    rescale=1./250, 
    vertical_flip=True)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(16,24), color_mode="rgb", shuffle = True, batch_size=batch_size, class_mode='categorical')
val_generator = train_datagen.flow_from_directory(val_dir, target_size=(16,24), batch_size=batch_size, class_mode='categorical')

# Compilation et entraînement du modèle
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, steps_per_epoch=train_generator.n//batch_size, epochs=epochs, validation_data=val_generator, validation_steps=val_generator.n//batch_size)

In [ ]:
# Courbe de la perte
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss')
plt.legend()

# Courbe de l'exactitude
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

La matrice de confusion est une matrice carrée qui représente la performance d'un modèle de classification. Elle est composée de différentes valeurs qui indiquent le nombre de prédictions correctes et incorrectes effectuées par le modèle pour chaque classe.
Les valeurs diagonales de la matrice représentent les prédictions correctes pour chaque classe. Plus ces valeurs sont élevées, meilleure est la performance du modèle.
Les valeurs non diagonales de la matrice représentent les prédictions incorrectes. Elles indiquent combien de fois le modèle a confondu une classe avec une autre.
En analysant les valeurs de la matrice de confusion, on peut déduire des informations sur les performances du modèle, telles que les classes les plus difficiles à prédire ou les confusions les plus fréquentes entre les classes.

In [ ]:
from sklearn.metrics import confusion_matrix

# Obtenir les prédictions pour le jeu de validation
predictions = model.predict(val_generator)
predicted_labels = np.argmax(predictions, axis=1)

# Obtenir les étiquettes réelles du jeu de validation
true_labels = val_generator.labels

# Créer la matrice de confusion
confusion_matrix = confusion_matrix(true_labels, predicted_labels)

# Afficher la matrice de confusion
print("Matrice de confusion :")
print(confusion_matrix)

In [ ]:
# Charger les données du fichier "test.npy"
data = np.load("/kaggle/input/navires-2023-la-mano/test.npy")
print(data.shape)

# Normaliser les données
normalized_data = data / 250.0

# Prédire les étiquettes
predictions = model.predict(normalized_data)

predicted_labels = np.argmax(predictions, axis=1)
print(predictions.shape)

results_df = pd.DataFrame({'Id': range(0, len(predicted_labels)), 'Category': predicted_labels})
print(results_df)

results_df.to_csv('submission.csv', index=False)

In [ ]:
#res = model.predict(np.load("/kaggle/input/navires-2023-la-mano/test.npy")).argmax(axis=1)
#df = pd.DataFrame({"Category":res})
#df.to_csv("submission.csv", index_label="Id")

In [ ]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink
FileLink(r'submission.csv')